In [1]:
import cpa
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad

[rank: 0] Global seed set to 0
2024-10-02 12:05:12,177	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-02 12:05:12,719	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
split = 2
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
try:
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)

adata

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

adata_ood.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]
adata_ood.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_ood.obs["cell_type"] = adata.obs["cell_type"]

adata_test.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]
adata_test.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_test.obs["cell_type"] = adata.obs["cell_type"]

adata_train.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]
adata_train.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_train.obs["cell_type"] = adata.obs["cell_type"]

In [5]:
control_cells_gex = adata_ood[adata_ood.obs["condition"]=="control"].X.A
obs_names = list(adata_ood[adata_ood.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_ood.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_ood[adata_ood.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_ood_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_ood_new.var = adata_ood.var


control_cells_gex = adata_test[adata_test.obs["condition"]=="control"].X.A
obs_names = list(adata_test[adata_test.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_test.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_test[adata_test.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_test_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_test_new.var = adata_test.var

In [6]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test_new, adata_ood_new), label="split", keys=["train", "test", "ood"], join="inner")

In [7]:
adata_combined

AnnData object with n_obs × n_vars = 61467 × 2000
    obs: 'condition', 'cell_type', 'condition_ID', 'log_dose', 'smiles_rdkit', 'split'

In [8]:
adata_test

AnnData object with n_obs × n_vars = 2900 × 2000
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type', 'cell_line', 'smiles_drug_1', 'smiles_drug_2', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'cov_drug_dose', 'condition_ID', 'log_dose', 'smiles_rdkit'
    var: 'id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Drug1_colors', 'Drug2_colors', 'Well_colors', 'condition_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'ood_1_colors', 'ood_2_colors', 'ood_3_colors', 'ood_4_colors', 'pathway1_colors', 'pathway2_colors', 'pathway_colors', 'pca', 'rank_genes_groups', 'rank_genes_groups_cov_all', 'split_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'ecfp_drug_

In [9]:
adata_test_new

AnnData object with n_obs × n_vars = 12000 × 2000
    obs: 'condition_ID', 'log_dose', 'smiles_rdkit', 'cell_type', 'condition'
    var: 'id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [10]:
adata_ood_new

AnnData object with n_obs × n_vars = 3500 × 2000
    obs: 'condition_ID', 'log_dose', 'smiles_rdkit', 'cell_type', 'condition'

In [11]:
adata_ood

AnnData object with n_obs × n_vars = 15011 × 2000
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type', 'cell_line', 'smiles_drug_1', 'smiles_drug_2', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'cov_drug_dose', 'condition_ID', 'log_dose', 'smiles_rdkit'
    uns: 'rank_genes_groups_cov_all'
    obsm: 'X_pca', 'X_umap', 'ecfp_drug_1', 'ecfp_drug_2'
    layers: 'counts'

In [12]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [13]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [14]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      deg_uns_cat_key='rank_genes_groups_cov',
                      max_comb_len=2,
                     )

100%|██████████| 61467/61467 [00:00<00:00, 994960.90it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [15]:
df_hsearch = pd.read_csv("result_hsearch.csv").T
df_hsearch.columns = df_hsearch.iloc[0]
df_hsearch = df_hsearch.drop(df_hsearch.index[0])
config = df_hsearch.sort_values("r_squared", ascending=False).iloc[0]
del config["r_squared"]
config = config.to_dict()

In [16]:
config_adapted = {}
for k,v in config.items():
    if k=="adv_loss":
        config_adapted[k] = v
    elif isinstance(v, float): 
        if np.isnan(v):
            config_adapted[k] = None
        else:
            raise ValueError
    elif v == "True":
        config_adapted[k] = True
    elif v == "False":
        config_adapted[k] = False
    else:
        if float(v) == int(float(v)):
            config_adapted[k] = int(float(v))
        else:
            config_adapted[k] = float(v)

In [17]:
trainer_params = config_adapted

In [18]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "gauss",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}


In [19]:
trainer_params

{'n_epochs_kl_warmup': None,
 'n_epochs_pretrain_ae': 50,
 'n_epochs_adv_warmup': 50,
 'n_epochs_mixup_warmup': 3,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'n_hidden_adv': 64,
 'n_layers_adv': 2,
 'use_batch_norm_adv': True,
 'use_layer_norm_adv': False,
 'dropout_rate_adv': 0.3,
 'reg_adv': 20,
 'pen_adv': 20,
 'lr': 0.0003,
 'wd': 4e-07,
 'adv_lr': 0.0003,
 'adv_wd': 4e-07,
 'adv_loss': 'cce',
 'doser_lr': 0.0003,
 'doser_wd': 4e-07,
 'do_clip_grad': False,
 'gradient_clip_value': 1,
 'step_size_lr': 10}

In [20]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split_1ct_MEC",
    train_split="train_train",
    valid_split="train_valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [21]:
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/1",
)

100%|██████████| 32/32 [00:00<00:00, 219.32it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/2000:   0%|          | 4/2000 [01:18<11:45:54, 21.22s/it, v_num=1, recon=-2.52e+3, r2_mean=0.79, adv_loss=2.46, acc_pert=0.168]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [03:56<16:22:44, 29.62s/it, v_num=1, recon=-3.04e+3, r2_mean=0.798, adv_loss=2.36, acc_pert=0.183, val_recon=-2.84e+3, disnt_basal=0.0739, disnt_after=0.0911, val_r2_mean=0.797, val_KL=nan]


Epoch 00009: cpa_metric reached. Module best state updated.


...

Epoch 30/2000:   1%|▏         | 29/2000 [14:32<16:59:41, 31.04s/it, v_num=1, recon=-3.6e+3, r2_mean=0.808, adv_loss=2.23, acc_pert=0.206, val_recon=-3.41e+3, disnt_basal=0.0597, disnt_after=0.0746, val_r2_mean=0.807, val_KL=nan] 


Epoch 00029: cpa_metric reached. Module best state updated.



disnt_basal = 0.05684330890073643
disnt_after = 0.07039273806828156
val_r2_mean = 0.8088086247140643
val_r2_var = 0.4687411411368656
Epoch 40/2000:   2%|▏         | 39/2000 [19:46<16:51:35, 30.95s/it, v_num=1, recon=-3.74e+3, r2_mean=0.81, adv_loss=2.24, acc_pert=0.206, val_recon=-3.44e+3, disnt_basal=0.0563, disnt_after=0.0691, val_r2_mean=0.812, val_KL=nan] 


Epoch 00039: cpa_metric reached. Module best state updated.



disnt_basal = 0.055992007698611776
disnt_after = 0.06945150080657327
val_r2_mean = 0.815096119608288
val_r2_var = 0.4567630913524666
Epoch 50/2000:   2%|▏         | 49/2000 [25:01<16:41:49, 30.81s/it, v_num=1, recon=-3.84e+3, r2_mean=0.813, adv_loss=2.22, acc_pert=0.216, val_recon=-3.53e+3, disnt_basal=0.0552, disnt_after=0.0662, val_r2_mean=0.815, val_KL=nan]
disnt_basal = 0.05430705424391541
disnt_after = 0.0658924883195812
val_r2_mean = 0.8150916377248084
val_r2_var = 0.45445599226824596
Epoch 60/2000:   3%|▎         | 59/2000 [30:26<17:24:14, 32.28s/it, v_num=1, recon=-3.91e+3, r2_mean=0.812, adv_loss=2.28, acc_pert=0.204, val_recon=-3.56e+3, disnt_basal=0.0526, disnt_after=0.0646, val_r2_mean=0.816, val_KL=nan]
disnt_basal = 0.05073281511449477
disnt_after = 0.062480619116961456
val_r2_mean = 0.8147690739417244
val_r2_var = 0.39259218285850445
Epoch 70/2000:   3%|▎         | 69/2000 [35:55<17:16:41, 32.21s/it, v_num=1, recon=-3.97e+3, r2_mean=0.814, adv_loss=2.47, acc_pert=0.169,

In [22]:
model.predict(adata_combined)

100%|██████████| 1921/1921 [00:06<00:00, 293.49it/s]


In [23]:
adata_combined.write(
    f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad"
)